In [1]:
# upto 20191012
# merge email first
# then calculate decile
# Remove 18+
# 90 segments in total
# The allocated store mapping is done throught the other code

# Write folder 1 day earlier

import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='BL_CRM_LR_Quarterly_upto20191012_2019Q4_'+str(datetime.datetime.now().date())+'.log', level=logging.INFO)
logging.info('Started')

samplerows=None

lastdate_date=datetime.date(2019,10,12) # Recent Saturday
active_Sunday=str(lastdate_date-datetime.timedelta(days=52*7-1))

lapsed_Sunday=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

            
# Write folder 1 day earlier
folder_write='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_'+str(datetime.datetime.now().date()-datetime.timedelta(days=1))+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
os.getcwd()

Lapsed Start on: 2018-04-15
Active Start on: 2018-10-14
Store Allocation Starting on: 2018-04-15
Last Saturday: 2019-10-12


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4'

In [ ]:
# Should run
df_transactions_18_months=pd.read_csv(folder_write+"BL_sales_trans_by_id_date_JL_"+str(datetime.datetime.now().date()-datetime.timedelta(days=1))+".csv")
df_transactions_18_months.head(4)




df_transactions_18_months_head100=df_transactions_18_months.head(100)

for col in df_transactions_18_months_head100.columns.tolist():
    print(df_transactions_18_months_head100[col].apply(lambda x: type(x)).unique())

In [4]:
print(df_transactions_18_months.shape)
df_transactions_18_months=df_transactions_18_months.rename(columns={"transaction_dt":"transaction_date"})
df_transactions_18_months.head(5)

(104765937, 4)


,customer_id_hashed,transaction_date,sales,trans
0,0000358fbc9f7162c1acbfc88f1211ebec1245400631f1...,2018-04-20,44.40,2
1,00004eb78a416ee5bf936f1fdb9d8504ea4694dae3aa95...,2018-04-16,3.00,1
2,00008158f7447208eeb00ca7bca86cccfa30dcbf90f2bc...,2018-04-16,39.50,1
3,00008a4e31d163efb872b8016ae17bde1df21b95ded478...,2018-04-17,5.00,1
4,00008e4bdadfb23fafd0805ad6f255dc4d9faf4e6c1129...,2018-04-17,44.14,1


In [5]:
###get recency
dfrecency=df_transactions_18_months[['customer_id_hashed','transaction_date']].sort_values("transaction_date",ascending=False).drop_duplicates()#Allready combined

print (min(dfrecency['transaction_date']))
print (max(dfrecency['transaction_date']))
dfrecency=dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency_id_with_last_transaction_date.csv',index=False)

2018-04-15
2019-10-12


In [6]:
dfrecency['transaction_date']=pd.to_datetime(dfrecency['transaction_date'])
dfrecency['recency']= lastdate_date - dfrecency['transaction_date']
dfrecency['recency']=dfrecency['recency'].apply(lambda x:x.days)
dfrecency['recency']=np.ceil((dfrecency['recency']+1)/30)
# recency as the np.floor month

dfrecency=dfrecency[['customer_id_hashed','recency','transaction_date']]
dfrecency=dfrecency.drop_duplicates('customer_id_hashed')
dfrecency=dfrecency.rename(columns={"transaction_date":"last_transaction_date"})
dfrecency.to_csv(folder_write + 'dfrecency_id_with_last_transaction_date_and_days_difference.csv',index=False)

dfrecency.shape

(21652333, 3)

In [7]:
print("df_transactions_18_months['customer_id_hashed'].nunique(): "+str(df_transactions_18_months['customer_id_hashed'].nunique()))
print("dfrecency['customer_id_hashed'].nunique(): "+str(dfrecency['customer_id_hashed'].nunique()))

dftotal=df_transactions_18_months.groupby('customer_id_hashed')['sales','trans'].sum().reset_index().rename(columns={"trans":"transactions"})
print("dftotal['customer_id_hashed'].nunique(): "+str(dftotal['customer_id_hashed'].nunique()))
print("dftotal.shape: "+str(dftotal.shape))

dftotal=pd.merge(dftotal,dfrecency,on='customer_id_hashed',how='outer')
print("dftotal.shape: "+str(dftotal.shape))


df_transactions_18_months['customer_id_hashed'].nunique(): 21652333
dfrecency['customer_id_hashed'].nunique(): 21652333
dftotal['customer_id_hashed'].nunique(): 21652333
dftotal.shape: (21652333, 3)
dftotal.shape: (21652333, 5)


In [8]:
del dfrecency
del df_transactions_18_months
gc.collect()

dftotal.shape

(21652333, 5)

In [9]:
dftotal.head(2)

,customer_id_hashed,sales,transactions,recency,last_transaction_date
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,3.80,1,2.0,2019-08-29
1,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,211.78,3,12.0,2018-10-30


In [10]:
# The blow is corrected, due to the ascending & decending for each KPI. The higher the better, or the lower the better
dftotal=dftotal.sort_values(['transactions','recency','sales'],ascending=[0,1,0])
dftotal.reset_index(drop=True, inplace=True)
dftotal.reset_index(inplace=True)
dftotal=dftotal.rename(columns={'index':'Transindex'})

dftotal=dftotal.sort_values(['sales','recency','transactions'],ascending=[0,1,0])
dftotal.reset_index(drop=True, inplace=True)
dftotal.reset_index(inplace=True)
dftotal=dftotal.rename(columns={'index':'Amtindex'})

dftotal=dftotal.sort_values(['recency','transactions','sales'],ascending=[1,0,0])
dftotal.reset_index(drop=True, inplace=True)
dftotal.reset_index(inplace=True)
dftotal=dftotal.rename(columns={'index':'recencyindex'})

c_ids=len(dftotal.index)
logging.info('total customers from transaction and amt: ')
logging.info(c_ids)
c_ids=np.ceil(c_ids/5.0)

dftotal['Transindex']=np.ceil((dftotal['Transindex']+1)/c_ids)
dftotal['Amtindex']=np.ceil((dftotal['Amtindex']+1)/c_ids)
dftotal['recencyindex']=np.ceil((dftotal['recencyindex']+1)/c_ids)

dftotal['RFM']=dftotal['recencyindex']*100 + dftotal['Transindex']*10 + dftotal['Amtindex']


In [11]:
dftotal.head(4)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0
2,1.0,1.0,1.0,ba00b08e0ecda5f850d28c55bcce69191bc8c24a811bb6...,77656.47,3765,1.0,2019-10-12,111.0
3,1.0,1.0,1.0,af1a67def27b7a4812c299f88c9128d1f1471df969ee7c...,75384.23,3710,1.0,2019-10-12,111.0


In [12]:
dftotal.to_csv(folder_write+"dftotal_with_index_all_EmailNotMergedYet.csv",index=False)

In [35]:
dftotal=pd.read_csv("./output_2019-10-20/dftotal_with_index_all_EmailNotMergedYet.csv")


In [36]:
print(dftotal.shape)

dftotal.head(2)

(21652333, 9)


,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0


# Merge in Emails, ZipLabels & Closest Stores

In [37]:
# Allocated shoppers stores to be added in !!!!!!!!!!!

In [38]:
dfiddetail=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-21',
                       usecols=['customer_id_hashed','email_address_hash','cleaned_zip_cd','closest_store','zip_type'],dtype=str)
print("dfiddetail.shape "+str(dfiddetail.shape))
print("dfiddetail['customer_id_hashed'].nunique() "+str(dfiddetail['customer_id_hashed'].nunique()))
print("dfiddetail['email_address_hash'].nunique() "+str(dfiddetail['email_address_hash'].nunique()))


dfiddetail.shape (30732606, 5)
dfiddetail['customer_id_hashed'].nunique() 30732606
dfiddetail['email_address_hash'].nunique() 30725689


In [39]:
gc.collect()

7

In [40]:
print("dftotal.shape "+str(dftotal.shape))
dftotal=pd.merge(dftotal,dfiddetail,on="customer_id_hashed",how="left")
print("dftotal.shape "+str(dftotal.shape))


dftotal.shape (21652333, 9)
dftotal.shape (21652333, 13)


In [41]:
dftotal.head(2)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684,trans_P
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926,trans_P


In [42]:
# Remove no email ids
print(dftotal.shape)
dftotal=dftotal[pd.notnull(dftotal['email_address_hash'])]
print(dftotal.shape)

(21652333, 13)
(19616323, 13)


In [44]:
gc.collect()
dftotal.head(2)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684,trans_P
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926,trans_P


In [46]:
dftotal['cleaned_zip_cd']=dftotal['cleaned_zip_cd'].replace("00nan","00000")
dftotal['zip_type']=dftotal['zip_type'].fillna("T")
dftotal['zip_type']=dftotal['zip_type'].replace("zips_10","S")
dftotal['zip_type']=dftotal['zip_type'].replace("trans_P","P")
dftotal['zip_type']=dftotal['zip_type'].replace("trans_S","S")
print(dftotal['zip_type'].unique())

['P' 'S' 'T']


In [47]:
dftotal.head(2)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684,P
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926,P


In [50]:
df_shopping_stores=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_allocated_store_by_id_shoppers_18_months_JL_20180415_20191012_JL_2019-10-20 19:46:00.089222.csv",
                              dtype=str,usecols=['customer_id_hashed','location_id'])

print(df_shopping_stores.shape)
print(df_shopping_stores['customer_id_hashed'].nunique())


(21589087, 2)
21589087


In [51]:
print(dftotal.shape)
dftotal=pd.merge(dftotal,df_shopping_stores,on="customer_id_hashed",how="left")
print(dftotal.shape)
dftotal=dftotal.rename(columns={"location_id":"shopper_store"})

(19616323, 13)
(19616323, 14)


In [54]:
dftotal.head(2)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type,shopper_store
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684,P,1684
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926,P,1926


In [55]:
dftotal['final_store_for_id']=np.where(pd.isnull(dftotal['shopper_store']),dftotal["closest_store"],dftotal['shopper_store'])
dftotal.shape

(19616323, 15)

In [66]:
print(dftotal[(pd.isnull(dftotal['final_store_for_id'])) & (dftotal['zip_type']!="T")].shape)

(0, 15)


In [62]:
dftotal.to_csv(folder_write+"final_details_by_id_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [63]:
gc.collect()

1035

In [65]:
dftotal.head(2)

,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type,shopper_store,final_store_for_id
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684,P,1684,1684
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926,P,1926,1926


# Add SOTF & Density

In [2]:
dftotal=pd.read_csv("./output_2019-10-20/final_details_by_id_JL_2019-10-21.csv")
dftotal['cleaned_zip_cd']=dftotal['cleaned_zip_cd'].astype(str)

In [3]:
test_value_type=dftotal.head(500).append(dftotal.tail(500))
for col in test_value_type.columns.tolist():
    print(col,test_value_type[col].apply(type).unique())

recencyindex [<class 'float'>]
Amtindex [<class 'float'>]
Transindex [<class 'float'>]
customer_id_hashed [<class 'str'>]
sales [<class 'float'>]
transactions [<class 'int'>]
recency [<class 'float'>]
last_transaction_date [<class 'str'>]
RFM [<class 'float'>]
email_address_hash [<class 'str'>]
cleaned_zip_cd [<class 'str'>]
closest_store [<class 'float'>]
zip_type [<class 'str'>]
shopper_store [<class 'float'>]
final_store_for_id [<class 'float'>]


In [4]:
dftotal.shape

(19616323, 15)

In [5]:
print("dftotal['last_transaction_date'].min(): "+str(dftotal['last_transaction_date'].min()))
print("active_Sunday: "+active_Sunday)
print("lapsed_Sunday: "+lapsed_Sunday)
dftotal['Recency_Status']=np.where(dftotal['last_transaction_date']>=active_Sunday,"Active (0-12)","Lapsed (13-18)")

dftotal.shape

dftotal['last_transaction_date'].min(): 2018-04-15
active_Sunday: 2018-10-14
lapsed_Sunday: 2018-04-15


(19616323, 16)

In [6]:
ExcelFile=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Copy of BL 2019Q4 Budget Allocation Model 20191018_JC_JL_20191020.xlsx")
ExcelFile.sheet_names

['parameters',
 'Closed 12 mo thru July (57)',
 "Closing Aug-Oct '19 (12)",
 'SOTF pre 11-15-19 (462)',
 'SOTF New,Relo Oct-Nov SOTF (11)',
 'all_PS10_zips_BL',
 '1_raw',
 '1_1',
 '2_raw_2018Q4filled',
 '3_sales%_budget',
 '4_SOTF',
 'Sheet2',
 '5_Rewards',
 '6_closed_Quad3',
 '7_split',
 'HML_PS_Zip 12 Mo',
 'HML_PS_Zip 18 mo',
 'Summary Chart 12 mo',
 'Summary Chart 18mo',
 'Zips by Density and PS 18mo',
 'Stores by Density 18mo',
 'Sheet1',
 '7_split 9mo V2']

In [7]:
# location names changed for the convience
df_store_density=ExcelFile.parse("Summary Chart 18mo",usecols=["location_id","Tier"],dtype=str)
df_store_density=df_store_density.rename(columns={"Tier":"Density_Tier","location_id":"final_store_for_id"})
df_store_density['final_store_for_id']=df_store_density['final_store_for_id'].astype(int)


df_SOTF=ExcelFile.parse("4_SOTF",usecols=["location_id","copy_SOTF_label"],dtype=str)
df_SOTF=df_SOTF.rename(columns={"copy_SOTF_label":"SOTF","location_id":"final_store_for_id"})
df_SOTF['SOTF']=df_SOTF['SOTF'].replace("nan",np.nan)
df_SOTF['final_store_for_id']=df_SOTF['final_store_for_id'].astype(int)

del ExcelFile
gc.collect()

89669

In [8]:
print(dftotal['zip_type'].unique())
dftotal_PS=dftotal[dftotal['zip_type']!="T"]
print(dftotal_PS.shape)
print(dftotal_PS[pd.isnull(dftotal_PS['final_store_for_id'])].shape)
del dftotal_PS
gc.collect()
# Remove the T after the decile scoring

dftotal['final_store_for_id']=dftotal['final_store_for_id'].fillna(0)
dftotal['final_store_for_id']=dftotal['final_store_for_id'].astype(int)

['P' 'S' 'T']
(17333538, 16)
(0, 16)


In [9]:
print(dftotal.shape)
dftotal=pd.merge(dftotal,df_store_density,on="final_store_for_id",how="left")
dftotal=pd.merge(dftotal,df_SOTF,on="final_store_for_id",how="left")
print(dftotal.shape)


(19616323, 16)
(19616323, 18)


In [10]:
dftotal[pd.isnull(dftotal['email_address_hash'])].shape

(0, 18)

In [11]:
print(dftotal['Density_Tier'].unique())
print(dftotal['SOTF'].unique())
print(dftotal[pd.isnull(dftotal['Density_Tier'])].shape)

['M' 'L' 'H' nan]
[nan 'SOTF']
(757408, 18)


In [12]:
# Revise the final_store_for_id if the stores are not in density defination file
# Use the nearest stores instead

dftotal['nearest_store']=dftotal['closest_store']
dftotal['nearest_store']=dftotal['nearest_store'].fillna(0)
dftotal['nearest_store']=dftotal['nearest_store'].astype(int)

df_store_density_copy=df_store_density.copy()
df_store_density_copy=df_store_density_copy.rename(columns={"final_store_for_id":"nearest_store","Density_Tier":"Density_2"})
dftotal=pd.merge(dftotal,df_store_density_copy,on="nearest_store",how="left")

dftotal['Density_Tier_Final']=np.where(pd.isnull(dftotal['Density_Tier']),dftotal['Density_2'],dftotal['Density_Tier'])

del dftotal['Density_Tier']
del dftotal['Density_2']
dftotal=dftotal.rename(columns={"Density_Tier_Final":"Density_Tier"})

In [13]:
# Also try to see the nearest store whether a SOTF or not
df_SOTF_copy=df_SOTF.copy()
df_SOTF_copy=df_SOTF_copy.rename(columns={"final_store_for_id":"nearest_store","SOTF":"SOTF_2"})
dftotal=pd.merge(dftotal,df_SOTF_copy,on="nearest_store",how="left")

dftotal['SOTF_Final']=np.where(pd.isnull(dftotal['SOTF']),dftotal['SOTF_2'],dftotal['SOTF'])
del dftotal['SOTF']
del dftotal['SOTF_2']
dftotal=dftotal.rename(columns={"SOTF_Final":"SOTF"})

In [14]:
dftotal['Density_Tier']=dftotal['Density_Tier'].fillna("M")
dftotal['SOTF']=dftotal['SOTF'].fillna("Legacy")

print(dftotal['Density_Tier'].unique())
print(dftotal['SOTF'].unique())

['M' 'L' 'H']
['Legacy' 'SOTF']


In [15]:
print(dftotal.shape)
dftotal.head(4)

(19616323, 19)


,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type,shopper_store,final_store_for_id,Recency_Status,nearest_store,Density_Tier,SOTF
0,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684.0,P,1684.0,1684,Active (0-12),1684,M,Legacy
1,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926.0,P,1926.0,1926,Active (0-12),1926,M,Legacy
2,1.0,1.0,1.0,ba00b08e0ecda5f850d28c55bcce69191bc8c24a811bb6...,77656.47,3765,1.0,2019-10-12,111.0,ff42f56047a09441444582bb3f23eec007869ea4aec58f...,1702,5194.0,P,5194.0,5194,Active (0-12),5194,M,Legacy
3,1.0,1.0,1.0,af1a67def27b7a4812c299f88c9128d1f1471df969ee7c...,75384.23,3710,1.0,2019-10-12,111.0,d48d9bde483e3a6524f75f41209df8b6848c1cd6714aff...,47250,255.0,P,255.0,255,Active (0-12),255,M,Legacy


In [16]:
dftotal.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_2019Q4Audience_details_18mo_shoppers_with_emails_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


# Deciles and Scoring

In [17]:
dftotal['cleaned_zip_cd']=dftotal['cleaned_zip_cd'].astype(str)

dftotal = dftotal.sort_values(['RFM','recency','transactions',
                               'sales'],ascending = [1,1,0,0])

print(dftotal.shape)
dftotal=dftotal.drop_duplicates("email_address_hash") ####### Email duplications also dropped
print("drop_duplicates('email_address_hash')",dftotal.shape) 

dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)

dftotal = dftotal.rename(columns = {'index':'frmindex'})
c_ids = len(dftotal.index)
c_ids = np.ceil(c_ids/10.0)
dftotal['frmindex'] = np.ceil((dftotal['frmindex']+1)/c_ids)

print("Done with the RFM scoring, "+str(datetime.datetime.now()))

dftotal['frmindex']=dftotal['frmindex'].apply(lambda x: "D"+str(int(float(x))).zfill(2))
dftotal['cleaned_zip_cd']=dftotal['cleaned_zip_cd'].apply(lambda x: x.zfill(5))

(19616323, 19)
drop_duplicates('email_address_hash') (19614845, 19)
Done with the RFM scoring, 2019-10-23 14:04:42.399116


In [18]:
df_H=pd.DataFrame({"frmindex":['D01','D02','D03','D04']})
df_H['HML_Group']="H"

df_M=pd.DataFrame({"frmindex":['D05','D06','D07']})
df_M['HML_Group']="M"

df_L=pd.DataFrame({"frmindex":['D08','D09','D10']})
df_L['HML_Group']="L"

df_HML=df_H.append(df_M).append(df_L)
gc.collect()

271

In [19]:
dftotal=pd.merge(dftotal,df_HML,on='frmindex',how="left")
dftotal.shape

(19614845, 21)

In [20]:
dftotal.head(4)

,frmindex,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,...,cleaned_zip_cd,closest_store,zip_type,shopper_store,final_store_for_id,Recency_Status,nearest_store,Density_Tier,SOTF,HML_Group
0,D01,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,...,28607,1684.0,P,1684.0,1684,Active (0-12),1684,M,Legacy,H
1,D01,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,...,79938,1926.0,P,1926.0,1926,Active (0-12),1926,M,Legacy,H
2,D01,1.0,1.0,1.0,ba00b08e0ecda5f850d28c55bcce69191bc8c24a811bb6...,77656.47,3765,1.0,2019-10-12,111.0,...,01702,5194.0,P,5194.0,5194,Active (0-12),5194,M,Legacy,H
3,D01,1.0,1.0,1.0,af1a67def27b7a4812c299f88c9128d1f1471df969ee7c...,75384.23,3710,1.0,2019-10-12,111.0,...,47250,255.0,P,255.0,255,Active (0-12),255,M,Legacy,H


## Experian Holiday Shoppers 

In [21]:
Experian_data=pd.read_table("/home/jian/BigLots/Experian/data_20191016/Experian_20191001114500.txt",
                              usecols=["CustomerNumber","MultiClusterDesc"],sep="|",dtype=str)
df_experian_mapping=pd.read_table("/home/jian/BigLots/Experian/data_20191016/Experian 2019 customer hashed.txt",
                                 dtype=str,sep="|")
df_experian_mapping=df_experian_mapping.rename(columns={"Customer_Num":"CustomerNumber"})

In [22]:
print(Experian_data.shape)
print(Experian_data['MultiClusterDesc'].unique())

Experian_data=Experian_data[Experian_data['MultiClusterDesc']=="Holiday Shoppers"]
print(Experian_data.shape)
Experian_data=pd.merge(Experian_data,df_experian_mapping,how="left",on="CustomerNumber")
print(Experian_data.shape)

(17626573, 2)
['Home Focused' 'Holiday Shoppers' 'Sporadic Seniors' 'Starters in Need'
 'Spending Fanatics' 'Urban Affluence' 'Infrequent Families'
 'Rising Hispanics' nan]
(1299463, 2)
(1299463, 3)


In [23]:
Experian_data['hashed_customer_num'].nunique()

1299463

In [24]:
dftotal.head(2)

,frmindex,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,...,cleaned_zip_cd,closest_store,zip_type,shopper_store,final_store_for_id,Recency_Status,nearest_store,Density_Tier,SOTF,HML_Group
0,D01,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,...,28607,1684.0,P,1684.0,1684,Active (0-12),1684,M,Legacy,H
1,D01,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,...,79938,1926.0,P,1926.0,1926,Active (0-12),1926,M,Legacy,H


In [25]:
dftotal['ExperianCluster']=np.where(dftotal['customer_id_hashed'].isin(Experian_data['hashed_customer_num'].tolist()),
                                    "HolidayShoppers","Others")
del Experian_data
del df_experian_mapping
gc.collect()

111

In [26]:
dftotal['segment_Decile_Active']=dftotal['Recency_Status']+"_"+dftotal['frmindex']+"_Density("+dftotal['Density_Tier']+")_"+\
                          dftotal['SOTF']+"_"+dftotal['ExperianCluster']+"_"+dftotal['zip_type']+"_2019Q4"
dftotal['segment_Decile_Lapsed']=dftotal['Recency_Status']+"_"+dftotal['frmindex']+"_Density("+dftotal['Density_Tier']+")_"+\
                      dftotal['SOTF']+"_"+"AllExpCluster"+"_"+dftotal['zip_type']+"_2019Q4"
dftotal['segment_Score_Active']=dftotal['Recency_Status']+"_Score("+dftotal['HML_Group']+")_Density("+dftotal['Density_Tier']+")_"+\
                          dftotal['SOTF']+"_"+dftotal['ExperianCluster']+"_"+dftotal['zip_type']+"_2019Q4"
dftotal['segment_Score_Lapsed']=dftotal['Recency_Status']+"_Score("+dftotal['HML_Group']+")_Density("+dftotal['Density_Tier']+")_"+\
                          dftotal['SOTF']+"_"+"AllExpCluster"+"_"+dftotal['zip_type']+"_2019Q4"
    
dftotal['segment']=np.where(dftotal['Recency_Status']=="Active (0-12)",dftotal['segment_Score_Active'],dftotal['segment_Score_Lapsed'])

In [27]:
dftotal.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_2019Q4_Final_Audience_details_18mo_shoppers_with_emails_seg_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
# Copy one file to the quarterly no shopper celery folder 
# "/home/jian/celery/Audience_No_Shop/Quarterly_Decile_Detail/df_detail_2019Q4.csv"

In [28]:
df_summary_0_18_score=dftotal.groupby(["segment","Recency_Status","HML_Group","Density_Tier","SOTF","ExperianCluster","zip_type"])['customer_id_hashed'].count().to_frame().reset_index()
df_summary_0_18_score=df_summary_0_18_score.rename(columns={"customer_id_hashed":"id_count"})

df_summary_0_18_decile=dftotal.groupby(["segment","Recency_Status","frmindex","Density_Tier","SOTF","ExperianCluster","zip_type"])['customer_id_hashed'].count().to_frame().reset_index()
df_summary_0_18_decile=df_summary_0_18_decile.rename(columns={"customer_id_hashed":"id_count"})


In [29]:
writer=pd.ExcelWriter("./BL_2019Q4_audience_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary_0_18_score.to_excel(writer,"by_HML_score",index=False)
df_summary_0_18_decile.to_excel(writer,"by_Decile_10",index=False)
writer.save()


In [30]:
pd.options.display.max_columns=30
dftotal.head(4)

,frmindex,recencyindex,Amtindex,Transindex,customer_id_hashed,sales,transactions,recency,last_transaction_date,RFM,email_address_hash,cleaned_zip_cd,closest_store,zip_type,shopper_store,final_store_for_id,Recency_Status,nearest_store,Density_Tier,SOTF,HML_Group,ExperianCluster,segment_Decile_Active,segment_Decile_Lapsed,segment_Score_Active,segment_Score_Lapsed,segment
0,D01,1.0,1.0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,76267.32,4922,1.0,2019-10-11,111.0,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,1684.0,P,1684.0,1684,Active (0-12),1684,M,Legacy,H,Others,Active (0-12)_D01_Density(M)_Legacy_Others_P_2...,Active (0-12)_D01_Density(M)_Legacy_AllExpClus...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...,Active (0-12)_Score(H)_Density(M)_Legacy_AllEx...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...
1,D01,1.0,1.0,1.0,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,69187.85,4167,1.0,2019-10-12,111.0,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,79938,1926.0,P,1926.0,1926,Active (0-12),1926,M,Legacy,H,Others,Active (0-12)_D01_Density(M)_Legacy_Others_P_2...,Active (0-12)_D01_Density(M)_Legacy_AllExpClus...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...,Active (0-12)_Score(H)_Density(M)_Legacy_AllEx...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...
2,D01,1.0,1.0,1.0,ba00b08e0ecda5f850d28c55bcce69191bc8c24a811bb6...,77656.47,3765,1.0,2019-10-12,111.0,ff42f56047a09441444582bb3f23eec007869ea4aec58f...,01702,5194.0,P,5194.0,5194,Active (0-12),5194,M,Legacy,H,Others,Active (0-12)_D01_Density(M)_Legacy_Others_P_2...,Active (0-12)_D01_Density(M)_Legacy_AllExpClus...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...,Active (0-12)_Score(H)_Density(M)_Legacy_AllEx...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...
3,D01,1.0,1.0,1.0,af1a67def27b7a4812c299f88c9128d1f1471df969ee7c...,75384.23,3710,1.0,2019-10-12,111.0,d48d9bde483e3a6524f75f41209df8b6848c1cd6714aff...,47250,255.0,P,255.0,255,Active (0-12),255,M,Legacy,H,Others,Active (0-12)_D01_Density(M)_Legacy_Others_P_2...,Active (0-12)_D01_Density(M)_Legacy_AllExpClus...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...,Active (0-12)_Score(H)_Density(M)_Legacy_AllEx...,Active (0-12)_Score(H)_Density(M)_Legacy_Other...


In [31]:
print(dftotal['customer_id_hashed'].nunique())
print(dftotal['email_address_hash'].nunique())
print(dftotal.shape)

19614845
19614845
(19614845, 27)


In [32]:
list_id_18_shoppers=dftotal['customer_id_hashed'].unique().tolist()
list_email_18_shoppers=dftotal['email_address_hash'].unique().tolist()
del dftotal

gc.collect()

478

In [34]:
len(list_id_18_shoppers)

19614845

In [35]:
len(list_email_18_shoppers)

19614845

# Lapsed 19-48

In [2]:
lastdate_date=datetime.date(2019,10,12) # Recent Saturday
active_Sunday=str(lastdate_date-datetime.timedelta(days=52*7-1))

lapsed_Sunday=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_48_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*4-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=
print("Beginning_48_months_ago:"+Beginning_48_months_ago)
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)



Lapsed Start on: 2018-04-15
Active Start on: 2018-10-14
Store Allocation Starting on: 2018-04-15
Last Saturday: 2019-10-12
Beginning_48_months_ago:2015-10-18


In [3]:
# 1 out of the 3

chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
df_lapsed_sales_ids=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk=chunk[(chunk['transaction_date']>=Beginning_48_months_ago) & (chunk['transaction_date']<Beginning_18_months_ago)]
    chunk=chunk.drop_duplicates()
    
    df_lapsed_sales_ids=df_lapsed_sales_ids.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())

del chunk

print("Earliest Date:" + str(df_lapsed_sales_ids['transaction_date'].min()))
print("Latest Date:" + str(df_lapsed_sales_ids['transaction_date'].max()))
del df_lapsed_sales_ids['transaction_date']
df_lapsed_sales_ids=df_lapsed_sales_ids.drop_duplicates()
print(df_lapsed_sales_ids.shape)
logging.info("Deduped: "+str(datetime.datetime.now()))

gc.collect()

1 2019-10-23 15:00:29.414070
2 2019-10-23 15:00:57.459272
3 2019-10-23 15:01:16.563207
4 2019-10-23 15:01:35.651320
5 2019-10-23 15:02:05.309754
6 2019-10-23 15:02:34.915563
7 2019-10-23 15:03:04.654926
8 2019-10-23 15:03:34.906106
9 2019-10-23 15:04:05.610180
10 2019-10-23 15:04:36.748218
11 2019-10-23 15:05:08.141758
12 2019-10-23 15:05:39.965752
13 2019-10-23 15:06:12.294753
14 2019-10-23 15:06:39.030806
15 2019-10-23 15:06:59.502027
Earliest Date:2016-06-26
Latest Date:2018-04-14
(19678528, 1)


189

In [4]:
# 2 out of the 3

lapsed_trans=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str)# .drop_duplicates() # Doesn't go to score at all, so no need to read all columns
lapsed_trans=lapsed_trans[(lapsed_trans['transaction_date']>=Beginning_48_months_ago) & (lapsed_trans['transaction_date']<Beginning_18_months_ago)]
del lapsed_trans['transaction_date']

df_lapsed_sales_ids=df_lapsed_sales_ids.append(lapsed_trans).drop_duplicates()
del lapsed_trans
gc.collect()

14

In [7]:
# 3 out of the 3

daily_sales_historical=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
daily_sales_historical=[x for x in daily_sales_historical if x.split("MediaStormDailySales_week_ending_")[1][:10]>=Beginning_48_months_ago]
daily_sales_historical=[x for x in daily_sales_historical if x.split("MediaStormDailySales_week_ending_")[1][:10]<Beginning_18_months_ago]
print(len(daily_sales_historical))

i_counter=0
list_historical_19_48=[]
for file in daily_sales_historical:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed']).drop_duplicates() # No date read, since will exclude the shoppers in 18 later
    df=df[pd.notnull(df['customer_id_hashed'])]
    list_historical_19_48.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())
df=pd.concat(list_historical_19_48).drop_duplicates()

df_lapsed_sales_ids=df_lapsed_sales_ids.append(df).drop_duplicates()
del df
gc.collect()

94
1 2019-10-23 15:21:58.365772
11 2019-10-23 15:24:37.237482
21 2019-10-23 15:28:01.997377
31 2019-10-23 15:31:30.753891
41 2019-10-23 15:34:51.507565
51 2019-10-23 15:38:04.375310
61 2019-10-23 15:40:22.961713
71 2019-10-23 15:43:06.272728
81 2019-10-23 15:45:50.536291
91 2019-10-23 15:48:27.247511


28

In [16]:
pd.options.display.max_columns=30
dftotal=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_2019Q4_Final_Audience_details_18mo_shoppers_with_emails_seg_JL_2019-10-23.csv")
list_unique_18_ids=dftotal['customer_id_hashed'].unique().tolist()
list_unique_18_emails=dftotal['email_address_hash'].unique().tolist()
print("dftotal.shape: "+str(dftotal.shape))
print("len(list_unique_18_ids): "+str(len(list_unique_18_ids)))
print("len(list_unique_18_emails): "+str(len(list_unique_18_emails)))


dftotal.shape: (19614845, 27)
len(list_unique_18_ids): 19614845
len(list_unique_18_emails): 19614845


In [24]:
df_lapsed_sales_ids=df_lapsed_sales_ids[~df_lapsed_sales_ids['customer_id_hashed'].isin(list_unique_18_ids)]

In [25]:
print(df_lapsed_sales_ids.shape)
dfiddetail=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-21',
                       usecols=['customer_id_hashed','email_address_hash','closest_store','zip_type'],dtype=str)
df_lapsed_sales_ids=pd.merge(df_lapsed_sales_ids,dfiddetail,on="customer_id_hashed",how="left")
print(df_lapsed_sales_ids.shape)

(7713304, 1)
(7713304, 4)


In [28]:
df_lapsed_sales_ids=df_lapsed_sales_ids[df_lapsed_sales_ids['zip_type']=="trans_P"]
df_lapsed_sales_ids['zip_type']=df_lapsed_sales_ids['zip_type'].replace("trans_P","P")
df_lapsed_sales_ids=df_lapsed_sales_ids[pd.notnull(df_lapsed_sales_ids['email_address_hash'])]
df_lapsed_sales_ids=df_lapsed_sales_ids.drop_duplicates("email_address_hash")
print("df_lapsed_sales_ids.shape: "+str(df_lapsed_sales_ids.shape))


df_lapsed_sales_ids.shape: (4645186, 4)


In [32]:
print("df_lapsed_sales_ids['customer_id_hashed'].nunique(): "+str(df_lapsed_sales_ids['customer_id_hashed'].nunique()))
print("df_lapsed_sales_ids['email_address_hash'].nunique(): "+str(df_lapsed_sales_ids['email_address_hash'].nunique()))


df_lapsed_sales_ids['customer_id_hashed'].nunique(): 4645186
df_lapsed_sales_ids['email_address_hash'].nunique(): 4645186


In [34]:
df_lapsed_sales_ids['closest_store'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [42]:
ExcelFile=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Copy of BL 2019Q4 Budget Allocation Model 20191018_JC_JL_20191020.xlsx")

df_store_density=ExcelFile.parse("Summary Chart 18mo",usecols=["location_id","Tier"],dtype=str)
df_store_density=df_store_density.rename(columns={"Tier":"Density_Tier","location_id":"closest_store"})
# df_store_density['final_store_for_id']=df_store_density['final_store_for_id'].astype(int)


df_SOTF=ExcelFile.parse("4_SOTF",usecols=["location_id","copy_SOTF_label"],dtype=str)
df_SOTF=df_SOTF.rename(columns={"copy_SOTF_label":"SOTF","location_id":"closest_store"})
df_SOTF['SOTF']=df_SOTF['SOTF'].replace("nan",np.nan)
# df_SOTF['final_store_for_id']=df_SOTF['final_store_for_id'].astype(int)

del ExcelFile
gc.collect()

89669

In [45]:
df_lapsed_sales_ids=pd.merge(df_lapsed_sales_ids,df_SOTF,on="closest_store",how="left")
df_lapsed_sales_ids['SOTF']=df_lapsed_sales_ids['SOTF'].fillna("Legacy")

df_lapsed_sales_ids=pd.merge(df_lapsed_sales_ids,df_store_density,on="closest_store",how="left")
df_lapsed_sales_ids['Density_Tier']=df_lapsed_sales_ids['Density_Tier'].fillna("M")


In [55]:
df_lapsed_sales_ids['segment']="Lapsed (19-48)_Density("+df_lapsed_sales_ids['Density_Tier']+")_"+df_lapsed_sales_ids['SOTF']+\
                                '_AllExpCluster_'+df_lapsed_sales_ids['zip_type']+"_2019Q4"

In [58]:
df_summary_19_48_score=df_lapsed_sales_ids.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_summary_19_48_score

,segment,customer_id_hashed
0,Lapsed (19-48)_Density(H)_Legacy_AllExpCluster...,496714
1,Lapsed (19-48)_Density(L)_Legacy_AllExpCluster...,3885
2,Lapsed (19-48)_Density(L)_SOTF_AllExpCluster_P...,182337
3,Lapsed (19-48)_Density(M)_Legacy_AllExpCluster...,2496144
4,Lapsed (19-48)_Density(M)_SOTF_AllExpCluster_P...,1466106


In [59]:
writer=pd.ExcelWriter("./BL_2019Q4_audience_summary_13_to_48_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary_19_48_score.to_excel(writer,"13_to_48_months",index=False)
writer.save()

In [60]:
df_lapsed_sales_ids.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_2019Q4_Final_Audience_details_19_to_48_Lapsed_shoppers_with_emails_seg_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
